In [0]:
%python
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("assignment").getOrCreate()

In [0]:
%python
data1=spark.read.csv("dbfs:/FileStore/tables/bank_customer.csv",inferSchema=True,header=True)
data2=spark.read.csv("dbfs:/FileStore/tables/bank_fd_account.csv",inferSchema=True,header=True)
data3=spark.read.csv("dbfs:/FileStore/tables/bank_rd_account.csv",inferSchema=True,header=True)
data4=spark.read.csv("dbfs:/FileStore/tables/bank_sb_account.csv",inferSchema=True,header=True)

In [0]:
%python
data1.createOrReplaceTempView("bank_customer")
data2.createOrReplaceTempView("bank_fd_account")
data3.createOrReplaceTempView("bank_rd_account")
data4.createOrReplaceTempView("bank_sb_account")

In [0]:
%sql
select * from bank_customer

CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE
1001,John,null,Smith,M,21-OCT-78,IND
1002,Franklin,J,Dang,M,21-OCT-78,IND
1003,Alice,M,Powell,F,09-APR-76,IND
1004,Ramesh,R,Narayan,M,08-DEC-71,IND
1005,Nancy,S,Taylor,F,17-AUG-79,IND
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND
1007,DealWell Co.,null,null,null,null,CORP
1008,Aman,K,Mathur,M,30-OCT-80,IND
1009,Smitha,null,Ramesh,F,03-MAR-76,IND
1010,Popular Enterprises,null,null,null,null,CORP


In [0]:
%sql
select * from bank_fd_account

ACCOUNT_NO,CUST_ID,START_DT,FD_TERM_MNTH,INITIAL_AMT,ACC_STATUS
5212340001,1002,22-JUN-00,30,15000,Closed
5212340002,1002,11-MAY-02,66,25000,Active
5212340003,1007,26-NOV-03,36,80555,Closed
5212340004,1003,29-NOV-04,60,30000,Active
5212340005,1009,23-JUN-04,120,45000,Active
5212340006,1001,26-SEP-05,24,45000,Active
5212340007,1010,18-OCT-05,84,15000000,Active
5212340008,1008,18-DEC-06,36,98765,Active
5212340009,1006,14-FEB-07,120,25000,Active
5212340010,1007,21-SEP-08,60,2500000,Active


In [0]:

%sql
select BF.CUST_ID, BC.CUST_FNAME,BF.INITIAL_AMT, BF.ACCOUNT_NO from bank_customer as BC 
join  bank_fd_account as BF on BF.CUST_ID= BC.CUST_ID 
where BF.INITIAL_AMT>25000 and BF.ACC_STATUS= 'Active' 
order by BF.INITIAL_AMT desc;

CUST_ID,CUST_FNAME,INITIAL_AMT,ACCOUNT_NO
1010,Popular Enterprises,15000000,5212340007
1007,DealWell Co.,2500000,5212340010
1008,Aman,98765,5212340008
1009,Smitha,45000,5212340005
1001,John,45000,5212340006
1003,Alice,30000,5212340004


In [0]:
%sql
select * from bank_rd_account


ACCOUNT_NO,RD_CUST_ID,RD_START_DT,RD_TERM_YRS,INSTALLMENT_AMT,CURR_BALANCE,ACC_STATUS
6000010001,1006,13-JUN-06,10,1000,75000,Active
6000010002,1003,26-NOV-06,6,3000,208000,Active
6000010003,1008,03-OCT-07,5,2500,148000,Active
6000010004,1009,29-APR-07,4,6000,0,Closed
6000010005,1001,17-FEB-08,1,10000,0,Closed
6000010006,1007,16-JUL-08,9,20000,1000012,Active
6000010007,1001,26-MAY-08,4,5000,0,Closed
6000010008,1010,18-DEC-09,7,50000,1550000,Active
6000010009,1007,20-APR-09,4,14000,560000,Active
6000010010,1010,08-MAY-10,7,25000,700000,Active


In [0]:
%sql
select * from bank_sb_account

ACCOUNT_NO,PRIMARY_CUST_ID,SECONDARY_CUST_ID,CURR_BAL_AMT,ACC_STATUS,START_DATE,END_DATE
1000012003,1005,1001,0.0,Closed,20-NOV-06,23-AUG-11
1000012004,1008,null,84924.0,Active,11-MAR-06,null
1000012005,1009,1004,209844.03,Active,20-NOV-06,null
1000012006,1004,null,0.0,Closed,12-SEP-07,12-SEP-10
1000012007,1003,null,12300.0,Inactive,09-DEC-08,null
1000012008,1005,null,10235.0,Active,23-MAY-09,null
1000012009,1006,null,73535.0,Active,10-OCT-10,null
1000012010,1004,null,54674.0,Active,09-OCT-11,null
1000012001,1001,null,25000.0,Active,06-SEP-04,null
1000012002,1002,null,31109.43,Active,16-DEC-05,null


In [0]:
%sql
select bc.cust_id,bc.cust_fname,bf.account_no,bf.initial_amt,br.curr_balance,bs.curr_bal_amt,(bf.initial_amt + br.curr_balance +bs.curr_bal_amt) as networth from bank_rd_account as br
join  bank_fd_account as bf on bf.cust_id=br.rd_cust_id
join bank_sb_account as bs on br.rd_cust_id=bs.primary_cust_id
join bank_customer as bc on bc.cust_id= br.rd_cust_id
where bf.ACC_STATUS='Active' and br.ACC_STATUS='Active' and bs.ACC_STATUS='Active' 
order by networth desc

cust_id,cust_fname,account_no,initial_amt,curr_balance,curr_bal_amt,networth
1008,Aman,5212340008,98765,148000,84924.0,331689.0
1006,Ahmad,5212340009,25000,75000,73535.0,173535.0


In [0]:
data1.display()

CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE
1001,John,null,Smith,M,21-OCT-78,IND
1002,Franklin,J,Dang,M,21-OCT-78,IND
1003,Alice,M,Powell,F,09-APR-76,IND
1004,Ramesh,R,Narayan,M,08-DEC-71,IND
1005,Nancy,S,Taylor,F,17-AUG-79,IND
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND
1007,DealWell Co.,null,null,null,null,CORP
1008,Aman,K,Mathur,M,30-OCT-80,IND
1009,Smitha,null,Ramesh,F,03-MAR-76,IND
1010,Popular Enterprises,null,null,null,null,CORP


In [0]:
from pyspark.sql.functions import when
data5=data1.withColumn("CUST_TYP_INDEX",when(data1.CUST_TYPE=='IND',1.0).when(data1.CUST_TYPE=='CORP',2).otherwise(data1.CUST_TYPE))

In [0]:
data5.display()

CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE,CUST_TYP_INDEX
1001,John,null,Smith,M,21-OCT-78,IND,1.0
1002,Franklin,J,Dang,M,21-OCT-78,IND,1.0
1003,Alice,M,Powell,F,09-APR-76,IND,1.0
1004,Ramesh,R,Narayan,M,08-DEC-71,IND,1.0
1005,Nancy,S,Taylor,F,17-AUG-79,IND,1.0
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND,1.0
1007,DealWell Co.,null,null,null,null,CORP,2
1008,Aman,K,Mathur,M,30-OCT-80,IND,1.0
1009,Smitha,null,Ramesh,F,03-MAR-76,IND,1.0
1010,Popular Enterprises,null,null,null,null,CORP,2


In [0]:
%sql
SELECT bf.CUST_ID, bf.ACCOUNT_NO, bf.INITIAL_AMT AS AMT, bf.ACC_STATUS FROM bank_fd_account as bf WHERE bf.ACC_STATUS = 'Active'
UNION
SELECT bank_rd.RD_CUST_ID, bank_rd.ACCOUNT_NO, bank_rd.CURR_BALANCE, bank_rd.ACC_STATUS FROM bank_rd_account as bank_rd WHERE bank_rd.ACC_STATUS = 'Active'
UNION
SELECT bank_sb.PRIMARY_CUST_ID, bank_sb.ACCOUNT_NO, bank_sb.CURR_BAL_AMT, bank_sb.ACC_STATUS FROM bank_sb_account as bank_sb WHERE bank_sb.ACC_STATUS = 'Active'
ORDER BY CUST_ID, ACCOUNT_NO

CUST_ID,ACCOUNT_NO,AMT,ACC_STATUS
1001,1000012001,25000.0,Active
1001,5212340006,45000.0,Active
1002,1000012002,31109.43,Active
1002,5212340002,25000.0,Active
1003,5212340004,30000.0,Active
1003,6000010002,208000.0,Active
1004,1000012010,54674.0,Active
1005,1000012008,10235.0,Active
1006,1000012009,73535.0,Active
1006,5212340009,25000.0,Active
